# Curve Fitting 2

In [ ]:
import pandas as pd, numpy as np
import matplotlib.pyplot as plt
import statsmodels.api as sm

### Review: OLS for linear and transformed-linear models

#### Example 1: 

In [ ]:
djia = pd.read_csv('climate-change-2016.csv')
djia.head(6)

In [ ]:
djia.plot(x='CO2ppm', y='global_temp_anomaly', kind='scatter')
ax = plt.gca()
ax.set_ylabel(r"Mean Global Temp Anomaly ($^\circ C$))", fontsize=16)
ax.set_xlabel(r"Atmospheric CO$_2$, (ppm)", fontsize=16)
plt.yticks(fontsize=18)
plt.xticks(fontsize=18)
plt.show()

In [ ]:
carbon_array = sm.add_constant(djia['CO2ppm'].values) # necessary to get the intercept
model = sm.OLS(djia['global_temp_anomaly'], carbon_array)
results = model.fit()
results.params

In [ ]:
results.summary()

In [ ]:
xs = np.arange(300, 420)
ys = results.params['x1'] * xs + results.params['const']
djia.plot(x='CO2ppm', y='global_temp_anomaly', kind='scatter')
plt.plot(xs, ys, linewidth=4, color = 'orange')
ax = plt.gca()
ax.set_ylabel(r"Mean Global Temp Anomaly ($^\circ C$))", fontsize=16)
ax.set_xlabel(r"Atmospheric CO$_2$, (ppm)", fontsize=16)
plt.yticks(fontsize=18)
plt.xticks(fontsize=18)
plt.show()

In [ ]:
1 - results.mse_resid / results.mse_total

In [ ]:
results.rsquared_adj

#### Example 2:

In [ ]:
djia.plot(x='DJIA', y='global_temp_anomaly', kind = 'scatter')
ax = plt.gca()
ax.set_ylabel(r"Mean Global Temp Anomaly ($^\circ C$))", fontsize=16)
ax.set_xlabel(r"Dow Jones Industrial Average (\$)", fontsize=16)
plt.xscale("log")
plt.yticks(fontsize=18)
plt.xticks(fontsize=18)
plt.show()

In [ ]:
X = sm.add_constant(np.log10(djia['DJIA'])) # necessary to get the intercept
model = sm.OLS(djia['global_temp_anomaly'], X)
results = model.fit()

In [ ]:
results.summary()

In [ ]:
xs = np.arange(600, 20000)
ys = results.params['DJIA'] * np.log10(xs) + results.params['const'] 
djia.plot(x='DJIA', y='global_temp_anomaly', kind = 'scatter')
plt.plot(xs, ys, linewidth=4, color = 'orange')
ax = plt.gca()
ax.set_ylabel(r"Mean Global Temp Anomaly ($^\circ C$))", fontsize=16)
ax.set_xlabel(r"Dow Jones Industrial Average (\$)", fontsize=16)
#plt.xscale("log")
plt.yticks(fontsize=18)
plt.xticks(fontsize=12)
plt.show()

<font color = green> __Correlation vs. causation:__

### p-values and statistical significance

<font color = green> __Statistical significance:__

<font color = green> __Null hypothesis:__

<font color = green> __p-value:__

<font color = green> __Statistic:__

<font color = green> __Test statistic:__

<font color = green> __Analysis of variance:__

<font color = black>
ANOVA $F$-statistic:

$$ \frac{\mathrm{MSE_{pred}}}{\mathrm{MSE_{res}}} $$

In [ ]:
# Calculate the F-statistic by hand
results.mse_model / results.mse_resid

### Nonlinear least squares

In [ ]:
df = pd.read_csv('data_79_17.csv', index_col=0)
df.head()
short_stack = pd.concat((df.loc[yr, :] for yr in range(1979, 1981)))
shortdf = pd.DataFrame(short_stack.values, columns=['Extent'], index=np.arange(len(short_stack.values)))
long_stack = pd.concat((df.loc[yr, :] for yr in range(1979, 1999)))
longdf = pd.DataFrame(long_stack.values, columns=['Extent'], index=np.arange(len(long_stack.values)))
shortdf.plot(marker = '.')

In [ ]:
shortdf.head(6)

In [ ]:
# Try a polynomial:
X = np.column_stack([shortdf.index.values ** i for i in range(3)]) # Fill in the range here to pick the degree
model = sm.OLS(shortdf['Extent'], X)
results = model.fit()
results.summary()

In [ ]:
xs = shortdf.index.values
ys = results.params['const'] \
    + results.params['x1'] * xs \
    + results.params['x2'] * xs ** 2 \
    # + results.params['x3'] * xs ** 3 \
    # + results.params['x4'] * xs ** 4 \
    # + results.params['x5'] * xs ** 5 
    # + results.params['x6'] * xs ** 6 
    
shortdf.plot(marker = '.')
plt.plot(xs, ys, color='orange')

In [ ]:
results.mse_resid**0.5

How does this work? OLS used the data matrix

$$ \left( \begin{array}{cc} 1 & x_0 \\ 1 & x_1 \\ \vdots & \vdots \\ 1 & x_{N-1} \end{array} \right) $$

to fit the model

$$ \hat y = b_0 + b_1 x $$

To augment this to a quadratic model, we augment the data matrix to 

$$ \left( \begin{array}{ccc} 1 & x_0 & x_0^2 \\ 1 & x_1 & x_1^2 \\ \vdots & \vdots & \vdots \\ 1 & x_{N-1} & x_{N-1}^2 \end{array} \right) $$

which results in the fit

$$ \hat y = b_0 + b_1 x + b_2 x^2$$

We're treating $x^2$ as if it were just another variable. In theory we could do this with any function of $x$ we could calculate, fitting a model like

$$ \hat y = b_0 + b_f f(x) + b_g g(x) + \ldots $$

with the data matrix

$$ \left( \begin{array}{cccc} 1 & f(x_0) & g(x_0) & \ldots \\ 1 & f(x_1) & g(x_1) & \ldots \\ \vdots & \vdots & \vdots & \\ 1 & f(x_{N-1}) & g(x_{N-1}) & \ldots \end{array} \right) $$

as long as the parameters $b_i$ are just coefficients multiplying "features" that we constructed ourselves. In the context of OLS you mostly see this done with polynomials (or logs), but some other ML techniques use a similar idea with wilder functions.

## <font color = blue> Exercises

In [ ]:
yields = pd.read_csv('yield.csv')

### Exercise 1

Make a scatterplot of `Yield` vs. `Temp` from the `yields` data frame. Does it look like a linear regression will fit well?

### Exercise 2

Use the technique above to fit the following kinds of models:
* linear
* quadratic
* cubic

Which model has the highest $R^2$? Lowest RMSE?

## Arbitrary function fitting

In [ ]:
from scipy.optimize import curve_fit

In [ ]:
# this is the function that we will be fitting to our points. 
# a, freq, phi, and c are the parameters that we will vary until we get the best fit.
def func(x, a, freq, phi, c):
    return a*np.sin(freq * (x - phi)) + c

In [ ]:
popt, pcov = curve_fit(func, shortdf.index, shortdf['Extent']) # popt = parameters optimized, pcov = covariance matrix -- we don't need this
rmse = (sum((func(x, *popt) - shortdf['Extent'][x])**2 for x in shortdf.index) / (len(shortdf.index - 4)))**0.5
rmse

In [ ]:
xs = shortdf.index.values
ys = func(shortdf.index.values, *popt)
shortdf.plot(marker = '.')
plt.plot(xs, ys, color='orange')

In [ ]:
popt

In [ ]:
popt, pcov = curve_fit(func, longdf.index, longdf['Extent'], p0 = [4, 1/60, 0, 12, 0]) # p0: starting guess
rmse = (sum((func(x, *popt) - longdf['Extent'][x])**2 for x in longdf.index) / (len(longdf.index) - 4))**0.5
rmse

In [ ]:
xs = shortdf.index.values
ys = func(shortdf.index.values, *popt)
shortdf.plot(marker = '.')
plt.plot(xs, ys, color='orange')

In [ ]:
popt

In [ ]:
xs = longdf.index.values
ys = func(longdf.index.values, *popt)
longdf.plot(marker = '.')
plt.plot(xs, ys, color='orange')

<font color = green> __How could we make this better?__